In [9]:
# from moccasin import setup_notebook

# setup_notebook()

In [10]:
from moccasin.config import get_active_network
active_netwrork = get_active_network()
print(active_netwrork.name)

eth-forked


In [11]:
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network
import boa 

STARTING_ETH_BALANCE = int(1000e18)
STARTING_WETH_BALANCE = int(1e18)
STARTING_USDC_BALANCE = int(100e6)

def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)

def _add_token_balance(usdc, weth):
    print(f"Starting balance of the weth: {weth.balanceOf(boa.env.eoa)}")
    weth.deposit(value = STARTING_WETH_BALANCE)
    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)

    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    print("After deposit:")
    print("ETH:", boa.env.get_balance(boa.env.eoa))
    print("WETH:", weth.balanceOf(boa.env.eoa))
    print("USDC:", usdc.balanceOf(boa.env.eoa))
    print(f"Name: {weth.name()}")

def setup_script() -> Tuple[ABIContract, ABIContract,ABIContract,ABIContract]:
    print("Starting setup script...")

    active_netwrork = get_active_network()

    usdc = active_netwrork.manifest_named("usdc")
    weth = active_netwrork.manifest_named("weth")

    if active_netwrork.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc,weth)

def moccasin_main():
    setup_script()

moccasin_main()

Starting setup script...
Starting balance of the weth: 2000000000000000000
After deposit:
ETH: 999000000000000000000
WETH: 3000000000000000000
USDC: 300000000
Name: Wrapped Ether


In [12]:
usdc = active_netwrork.manifest_named("usdc")
weth = active_netwrork.manifest_named("weth")

In [14]:
from moccasin.config import get_config
config= get_config()
config.reload()
active_netwrork = config.get_active_network()
aavev3_pool_addresses_provider = active_netwrork.manifest_named("aavev3_pool_addresses_provider")
pool_address = aavev3_pool_addresses_provider.getPool()
print(pool_address)

0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [17]:
config.reload()
active_netwrork = config.get_active_network()
pool_contract = active_netwrork.manifest_named("pool", address = pool_address)

In [19]:
REFFERAL_CODE = 0
def deposit(pool_contract, token, amount):
    allowed_amout = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amout < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {token.name()} into Aave contract {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFFERAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)
if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)

(totalCollateralBase,   
totalDebtBase,   
availableBorrowsBase,
currentLiquidationThreshold, 
ltv,    
healthFactor) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"User Account Data \n totalCollateralBase = {totalCollateralBase}, \n totalDebtBase = {totalDebtBase}, \n availableBorrowsBase = {availableBorrowsBase} \n currentLiquidationThreshold = {currentLiquidationThreshold} \n ltv = {ltv} \n healthFactor = {healthFactor}")



User Account Data 
 totalCollateralBase = 795747514812, 
 totalDebtBase = 0, 
 availableBorrowsBase = 638905679643 
 currentLiquidationThreshold = 8281 
 ltv = 8029 
 healthFactor = 115792089237316195423570985008687907853269984665640564039457584007913129639935
